In [1]:
import sys
sys.path.append('../../../')
sys.path.append('/home/sheida.rahnamai/GIT/HDN/')
import torch
import pickle
import matplotlib.pyplot as plt
import numpy as np
from tifffile import imread
import glob
import matplotlib.patches as patches
import hdbscan as hd
from hdbscan import prediction
from sklearn.cluster import HDBSCAN
from collections import Counter
from openTSNE import TSNE
from openTSNE import TSNEEmbedding
from openTSNE import affinity
from openTSNE import initialization
from sklearn.decomposition import PCA
from scipy.spatial.distance import cdist
# %reload lib.plotting 
from lib import plotting as p
from lib import dataprep as dp
import random
import umap
from tqdm import tqdm
from lib.evaluation import FeatureExtractor
# from lib.dataloader import CustomTestDataset
from lib.dataloader import CustomDataset
import tifffile as tiff
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
import os
from boilerplate import boilerplate
from importlib import reload
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as mcolors
import seaborn as sns
import pandas as pd
from lib.logging import ClassType
from sklearn.metrics import confusion_matrix, adjusted_rand_score, normalized_mutual_info_score, homogeneity_score, completeness_score, v_measure_score
from boilerplate import boilerplate as blp
from sklearn.metrics import f1_score

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

patch_size = 64
sample_size = 300
centre_size = 4
n_channel = 32
hierarchy_level = 3
pad_size = (patch_size - centre_size) // 2
model_dir = "/group/jug/Sheida/HVAE/v01/model/"
# model_dir = "/home/sheida.rahnamai/GIT/HDN/examples/Pixel_Noise/Convallaria/testmodel/"
model = torch.load(model_dir+"HVAE_best_vae.net")

In [3]:
# train data

data_dir = "/localscratch/data/"

data_mean = model.data_mean.cpu().numpy()
data_std = model.data_std.cpu().numpy()


One_test_image = ['high_c4']

# Load test image
test_img_path = os.path.join(data_dir, One_test_image[0], f"{One_test_image[0]}_source.tif")
test_image = tiff.imread(test_img_path)

# Print loaded test images paths
print("Test image loaded from path:")
print(test_img_path)

# Load test ground truth images
test_gt_path = os.path.join(data_dir, One_test_image[0], f"{One_test_image[0]}_gt.tif")
test_ground_truth_image = tiff.imread(test_gt_path)

#test data

test_images = (test_image-data_mean)/data_std



Test image loaded from path:
/localscratch/data/high_c4/high_c4_source.tif


In [4]:
seg = tiff.imread('/group/jug/Sheida/pancreatic beta cells/download/seg/seuclidean/high_c4_pred.tif')

In [5]:
print('test image shape: ', test_images.shape, 'segmentation shape: ', seg.shape)

test image shape:  (1022, 545, 1082) segmentation shape:  (1022, 482, 1019)


In [6]:
_, x, y = seg.shape
_, v, w = test_ground_truth_image.shape
cropped_gt = test_ground_truth_image[:,30:30+x,30:30+y]

In [7]:
cropped_gt.shape

(1022, 482, 1019)

In [8]:
not_appeared = [0,0,0,0]
def dice_coefficient_per_class(y_true, y_pred, class_label):
    global not_appeared
    intersection = np.sum((y_true == class_label) & (y_pred == class_label))
    denominator = np.sum(y_true == class_label) + np.sum(y_pred == class_label)
    
    # Check if the denominator is zero to avoid division by zero
    if denominator == 0:
        not_appeared[class_label] += 1
        return 0.0  # or you can use np.nan if you want to mark it as missing
    return 2 * intersection / denominator

def dice_coefficient_multilabel(y_true, y_pred, num_classes):
    dice_scores = []
    for class_label in range(num_classes):
        dice = dice_coefficient_per_class(y_true, y_pred, class_label)
        dice_scores.append(dice)
    return np.array(dice_scores)

num_classes = 4  # Number of labels/classes (excluding the -1 label which is for outside of the class)


In [9]:
dsc_per_class = [0,0,0,0]
bg = 0
for i in tqdm(range(seg.shape[0])):
    unique_label = np.unique(cropped_gt[i])
    if -1 in unique_label and len(unique_label) == 1:
        bg += 1
        #continue
    dsc_per_class += dice_coefficient_multilabel(cropped_gt[i], seg[i], num_classes)

for i in range(num_classes):
    dsc_per_class[i] /= (seg.shape[0]-not_appeared[i])
print(f"Dice Similarity Coefficient per class: {dsc_per_class}")
print(f"Mean Dice Similarity Coefficient: {np.mean(dsc_per_class):.4f}")

  0%|          | 0/1022 [00:00<?, ?it/s]

100%|██████████| 1022/1022 [00:09<00:00, 102.42it/s]

Dice Similarity Coefficient per class: [0.87224199 0.74838928 0.70292081 0.57903117]
Mean Dice Similarity Coefficient: 0.7256


# correlation
Dice Similarity Coefficient per class: [0.87221151 0.47524653 0.70252993 0.57664873]<br>
Mean Dice Similarity Coefficient: 0.6567<br>
# cosine
Dice Similarity Coefficient per class: [0.87221153 0.4747142  0.70253007 0.57664789]<br>
Mean Dice Similarity Coefficient: 0.6565<br>
# euclidean
Dice Similarity Coefficient per class: [0.87218948 0.60899397 0.70190738 0.57710773]<br>
Mean Dice Similarity Coefficient: 0.6900<br>
# seuclidean
Dice Similarity Coefficient per class: [0.87224199 0.74838928 0.70292081 0.57903117]<br>
Mean Dice Similarity Coefficient: 0.7256<br>